# Large Survey DataBase

LSDB (Large Survey DataBase) is a framework designed for fast spatial analysis of extremely large astronomical catalogs, enabling efficient querying and crossmatching of over a billion sources. It specifically addresses large-scale data processing challenges, particularly those posed by LSST. Read more about LSDB here.
LSDB is a framework to facilitate and enable spatial analysis for extremely large astronomical databases (i.e. querying and crossmatching O(1B) sources). This package uses dask to parallelize operations across multiple HiPSCat partitioned surveys. 


Please check out:

 - [github.com/astronomy-commons/lsdb](github.com/astronomy-commons/lsdb)
 - [lsdb.io](lsdb.io) (Check out this! There are multiple links to already partitioned catalogs, like GaiaDR3, ZTF and others).

We are applying on S-PLUS a new server side application with lsdb. It allows adding constrains on queries over the network, like selecting specific columns, and filtering results. All the examples below follow this implementation.

We have a walkthrough on how to use LSDB with S-PLUS data available at [splus.cloud/documentation/lsdb_query](https://splus.cloud/documentation/lsdb_query).


### Lets import splusdata and lsdb

In [1]:
import lsdb
import splusdata

Using splusdata we can print the available catalogs, and get the link for one in particular.

In [2]:
# This will print all available links
print(splusdata.get_hipscats())

# This will get the link for a specific pattern
print(splusdata.get_hipscats("dr4/dual"))


{'dr4_vacs': {'qso_z': {'hipscats': ['qso_z'], 'margins': ['qso_z_2arcsec']}, 'photozs': {'hipscats': ['photozs'], 'margins': ['photozs_2arcsec']}, 'calib_flag': {'hipscats': ['calib_flag'], 'margins': ['calib_flag_2arcsec']}, 'sqg': {'hipscats': ['sqg'], 'margins': ['sqg_2arcsec']}}, 'sdr1': {'hipscats': ['shorts'], 'margins': ['shorts_2arcsec']}, 'dr4': {'hipscats': ['single', 'dual', 'psf'], 'margins': ['psf_2arcsec', 'single_2arcsec', 'dual_2arcsec']}}
[['https://splus.cloud/HIPS/catalogs/dr4/dual/', 'https://splus.cloud/HIPS/catalogs/dr4/dual_2arcsec/']]


### Important

Before getting into the code, start a dask Client with the desired specs, this is what does the "parallel magic". This step is very important. It will also print a url to a dashboard that you can use to check the status of your queries.

In [3]:
from dask.distributed import Client

client = Client(n_workers=10, memory_limit="8GB")
client

/Users/gustavoschwarz/miniconda3/lib/python3.12/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 50909 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:50909/status,
Dashboard: http://127.0.0.1:50909/status,Workers: 10
Total threads: 20,Total memory: 74.51 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:50910,Workers: 10
Dashboard: http://127.0.0.1:50909/status,Total threads: 20
Started: Just now,Total memory: 74.51 GiB
Comm: tcp://127.0.0.1:50936,Total threads: 2
Dashboard: http://127.0.0.1:50937/status,Memory: 7.45 GiB
Nanny: tcp://127.0.0.1:50913,


### Exploring the data

Before we start querying the data, let's take a look at the available columns in the catalog. 
This is important to know what we can query and what we can get from the database. Filtering the columns will make the query faster and more efficient.

In [4]:
# This will return a list because there may be more than
# one match for dr4/dual
dr4_dual_link = splusdata.get_hipscats("dr4/dual")[0]

# dr4_dual_link will be a tuple, because of the margin link.
dual_link = lsdb.read_hipscat(dr4_dual_link[0])

In [5]:
list(dual_link.columns)

['ID',
 'RA',
 'DEC',
 'A',
 'B',
 'BACKGROUND',
 'BACKGROUND_J0378',
 'BACKGROUND_J0395',
 'BACKGROUND_J0410',
 'BACKGROUND_J0430',
 'BACKGROUND_J0515',
 'BACKGROUND_J0660',
 'BACKGROUND_J0861',
 'BACKGROUND_g',
 'BACKGROUND_i',
 'BACKGROUND_r',
 'BACKGROUND_u',
 'BACKGROUND_z',
 'CLASS_STAR',
 'DET_ID_dual',
 'EBV_SCH',
 'ELLIPTICITY',
 'ELONGATION',
 'FLUX_RADIUS_20',
 'FLUX_RADIUS_50',
 'FLUX_RADIUS_70',
 'FLUX_RADIUS_90',
 'FWHM',
 'FWHM_J0378',
 'FWHM_J0395',
 'FWHM_J0410',
 'FWHM_J0430',
 'FWHM_J0515',
 'FWHM_J0660',
 'FWHM_J0861',
 'FWHM_g',
 'FWHM_i',
 'FWHM_n',
 'FWHM_n_J0378',
 'FWHM_n_J0395',
 'FWHM_n_J0410',
 'FWHM_n_J0430',
 'FWHM_n_J0515',
 'FWHM_n_J0660',
 'FWHM_n_J0861',
 'FWHM_n_g',
 'FWHM_n_i',
 'FWHM_n_r',
 'FWHM_n_u',
 'FWHM_n_z',
 'FWHM_r',
 'FWHM_u',
 'FWHM_z',
 'Field',
 'ID_DEC',
 'ID_RA',
 'ISOarea',
 'J0378_ID_dual',
 'J0378_PStotal',
 'J0378_aper_3',
 'J0378_aper_6',
 'J0378_auto',
 'J0378_iso',
 'J0378_petro',
 'J0395_ID_dual',
 'J0395_PStotal',
 'J0395_ape

### Querying the data

Here we will show some examples of queries that can be done with LSDB. We will show how to filter the columns and how to filter the data.

First we will instantiate the LSDB object with the catalog we want to query. Then we will use a `conesearch` or `box_search` method to get the data we want. 

Let's start with DR4.

In [6]:
dr4_links = splusdata.get_hipscats("dr4/dual")[0]

dual_margin = lsdb.read_hipscat(dr4_links[1])
dual = lsdb.read_hipscat(
    dr4_links[0],
    margin_cache=dual_margin,
    columns=["ID", "RA", "DEC", 
             "r_auto", "g_auto", "i_auto", "z_auto", "u_auto",
             "J0378_auto", "J0395_auto", "J0410_auto", "J0430_auto",
             "J0515_auto", "J0660_auto", "J0861_auto", 
             "e_r_auto", "e_g_auto", "e_i_auto", "e_z_auto", "e_u_auto",
             "e_J0378_auto", "e_J0395_auto", "e_J0410_auto", "e_J0430_auto",
             "e_J0515_auto", "e_J0660_auto", "e_J0861_auto",
            ],
    filters=[("r_auto", "<", 18), ("g_auto", ">", 14)],
)

In [7]:
dual.cone_search(
    23, # ra 
    1, # dec
    1*3600 # radius in arcseconds
).compute()

,ID,RA,DEC,r_auto,g_auto,i_auto,z_auto,u_auto,J0378_auto,J0395_auto,...,e_i_auto,e_z_auto,e_u_auto,e_J0378_auto,e_J0395_auto,e_J0410_auto,e_J0430_auto,e_J0515_auto,e_J0660_auto,e_J0861_auto
_hipscat_index,,,,,,,,,,,,,,,,,,,,,
5019946696994455552,DR4_3_STRIPE82-0036_0001237,23.32348,0.095314,17.327717,18.424343,16.738289,16.540657,99.0,18.166468,17.891554,...,8.200325,10.848548,99.0,25.081669,9.85723,8.928188,13.142833,7.773458,2.291646,2.814814
5019946701838876672,DR4_3_STRIPE82-0036_0001322,23.325101,0.098247,17.901476,19.286633,17.044563,16.800449,21.013481,18.13236,18.208101,...,0.258864,0.31434,1.670517,2.295334,3.203007,1.861223,5.198153,2.669804,0.440602,1.582434
5019946708080001024,DR4_3_STRIPE82-0036_0001236,23.323378,0.099017,17.117977,18.252464,16.241793,15.727602,99.0,17.698477,17.193613,...,9.044808,6.483394,99.0,12.782671,17.03096,19.169289,10.562528,21.115028,2.298365,2.947276
5019946710097461248,DR4_3_STRIPE82-0036_0001326,23.323143,0.100581,17.169542,18.385048,16.150099,15.568332,20.059423,18.034189,17.510506,...,11.065784,5.720607,91.388313,28.023588,24.20389,16.609213,19.296499,18.856331,3.068845,4.32406
5019946711703879680,DR4_3_STRIPE82-0036_0001405,23.323768,0.10228,16.834469,18.188778,15.874294,15.155374,21.012041,17.529688,17.14897,...,6.901323,3.572552,275.151642,11.749881,22.642403,12.225708,14.945191,19.872906,4.504867,3.686914
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5117251849790947328,DR4_3_STRIPE82-0036_0014055,23.798556,1.42368,15.018927,16.35486,13.913931,13.460277,19.127176,18.578876,18.478643,...,0.001925,0.001826,0.126758,0.10887,0.176334,0.083651,0.046338,0.015145,0.002922,0.002957
5117252874790764544,DR4_3_STRIPE82-0036_0014359,23.696787,1.410547,17.243887,17.872732,17.049198,17.044365,19.217907,19.070717,18.820202,...,0.018896,0.027464,0.110544,0.136397,0.192138,0.182212,0.082478,0.0458,0.016683,0.038205
5117253173123219456,DR4_3_STRIPE82-0036_0014227,23.656134,1.415943,15.969511,16.412558,15.846977,15.846822,17.533047,17.2722,17.267157,...,0.008859,0.012727,0.033319,0.037469,0.064947,0.053215,0.027667,0.017009,0.007793,0.017856


In [8]:
dual.box_search(
    [-1, 1], # start RA, end RA
    [-1, 1] # start Dec, end Dec
).compute()

,ID,RA,DEC,r_auto,g_auto,i_auto,z_auto,u_auto,J0378_auto,J0395_auto,...,e_i_auto,e_z_auto,e_u_auto,e_J0378_auto,e_J0395_auto,e_J0410_auto,e_J0430_auto,e_J0515_auto,e_J0660_auto,e_J0861_auto
_hipscat_index,,,,,,,,,,,,,,,,,,,,,
4899208249879822336,DR4_3_STRIPE82-0003_0011778,0.964254,-0.984355,16.980633,17.421703,16.841064,16.79199,18.530684,18.187626,18.122524,...,0.00798,0.012514,0.031748,0.040898,0.058372,0.035206,0.031451,0.020713,0.008493,0.017996
4899211249155309568,DR4_3_STRIPE82-0003_0012430,0.97616,-0.96809,15.041121,15.501884,14.899343,14.86096,16.781298,16.417391,16.413391,...,0.002211,0.002966,0.00855,0.010741,0.015406,0.008941,0.008149,0.005445,0.002564,0.004338
4899211261645946880,DR4_3_STRIPE82-0003_0012469,0.97898,-0.962708,16.135672,16.698135,15.943693,15.875214,18.214027,17.91004,17.767872,...,0.004219,0.006076,0.024999,0.033414,0.044294,0.022485,0.021069,0.01175,0.004729,0.008889
4899278485723283456,DR4_3_STRIPE82-0001_0011810,0.276649,-0.991464,15.448023,15.887675,15.306561,15.257038,17.019674,16.641893,16.630896,...,0.002926,0.004341,0.00967,0.008621,0.011946,0.008025,0.007473,0.007033,0.003136,0.00555
4899280014878441472,DR4_3_STRIPE82-0001_0012228,0.402118,-0.977255,17.930283,18.907173,17.592834,17.325251,20.071085,19.379517,19.606922,...,0.029493,0.044275,0.209014,0.139163,0.25514,0.262167,0.178927,0.10255,0.02909,0.061301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5477015167298437120,DR4_3_STRIPE82-0002_0044530,359.760099,0.998284,14.808131,15.016464,14.793859,14.838316,16.174797,15.619889,15.43687,...,0.001968,0.002702,0.005707,0.006154,0.007624,0.0058,0.005412,0.004045,0.002185,0.003659
5477019443450085376,DR4_3_STRIPE82-0002_0040702,359.543212,0.988262,16.676786,17.273762,16.509445,16.448231,18.74291,18.595207,18.740599,...,0.010684,0.01796,0.072295,0.09905,0.169153,0.063437,0.054186,0.026395,0.009642,0.022483
5477019576694734848,DR4_3_STRIPE82-0002_0045080,359.581452,0.997842,16.178837,16.673311,16.02849,15.98457,17.960754,17.596251,17.603821,...,0.00401,0.005727,0.017349,0.020359,0.029293,0.017549,0.016047,0.009619,0.004294,0.007417


### Matching hipscat tables

Assuming you have the the table from the example before, lets instantiate a new hipscat object pointing to another table, in this case Star Galaxy Quasar (sqg) separation table, and make a match with the dual catalog.

First lets instantiate the hipscat that we want to match with the dual catalog. In this case we are using the sqg table.

In [9]:
dr4_sqg = splusdata.get_hipscats("dr4/sqg")[0]
sqg_margin = lsdb.read_hipscat(dr4_sqg[1])
sqg = lsdb.read_hipscat(
    dr4_sqg[0],
    margin_cache=sqg_margin,
    filters=[("PROB_GAL", ">", 0.8)]
)

/Users/gustavoschwarz/miniconda3/lib/python3.12/site-packages/hipscat/io/file_io/file_io.py:320: UserWarning: point_map.fits file written in RING ordering, due to https://github.com/astronomy-commons/hipscat/issues/271. Converting to NESTED.
  warnings.warn(


Then we can use the match method to match the sqg table with the dual catalog. The match method will return a new hipscat object with the matched sources.

In [10]:
dual_sqg = sqg.crossmatch(
    dual, radius_arcsec = 1
)

In [11]:
result_table = dual_sqg.cone_search(
    1, #ra
    1, #dec
    1 * 3600 # radius in arcsecs
).compute()


In [12]:
result_table

,ID_sqg,RA_sqg,DEC_sqg,CLASS_sqg,model_flag_sqg,PROB_QSO_sqg,PROB_STAR_sqg,PROB_GAL_sqg,Norder_sqg,Dir_sqg,...,e_z_auto_dual,e_u_auto_dual,e_J0378_auto_dual,e_J0395_auto_dual,e_J0410_auto_dual,e_J0430_auto_dual,e_J0515_auto_dual,e_J0660_auto_dual,e_J0861_auto_dual,_dist_arcsec
_hipscat_index,,,,,,,,,,,,,,,,,,,,,
5091982615334879232,DR4_3_STRIPE82-0003_0027945,1.150865,0.028838,2.0,0,0.0,0.0,1.0,5,0,...,0.027895,0.123026,0.173134,0.120886,0.180917,0.116697,0.078257,0.023983,0.037479,0.000282
5091982615334879233,DR4_3_STRIPE82-0004_0000309,1.150864,0.028841,2.0,0,0.0,0.01,0.99,5,0,...,0.032874,99.0,2.064876,99.0,0.747462,1.903625,0.175603,0.030862,0.043016,0.000282
5091984044355223552,DR4_3_STRIPE82-0003_0032189,1.204843,0.022827,2.0,0,0.0,0.0,1.0,5,0,...,0.045413,0.112293,0.187618,0.170856,0.139337,0.158431,0.089865,0.034123,0.055606,0.000282
5091985502148493312,DR4_3_STRIPE82-0004_0002515,1.293757,0.09462,2.0,0,0.0,0.0,1.0,5,0,...,0.024065,0.281035,0.280912,0.26927,0.482463,0.248988,0.067966,0.020964,0.032753,0.000282
5091996295271284736,DR4_3_STRIPE82-0003_0041728,0.935803,0.038474,2.0,1,0.0,0.03,0.97,5,0,...,10.198874,99.0,99.0,99.0,318.049927,99.0,28.84516,3.151252,8.304696,0.000282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5477516506839384064,DR4_3_STRIPE82-0004_0031617,1.408031,1.376836,2.0,0,0.0,0.0,1.0,1,0,...,0.050726,3.499548,0.209596,0.211616,0.198783,0.159511,0.085544,0.037517,0.058727,0.000282
5477522346325573632,DR4_3_STRIPE82-0004_0030245,1.605574,1.412259,2.0,0,0.0,0.0,1.0,1,0,...,0.05859,0.787121,0.430592,0.403952,0.349742,0.250733,0.127288,0.04425,0.072602,0.000282
5477523454678794240,DR4_3_STRIPE82-0004_0030129,1.515884,1.415197,2.0,0,0.0,0.0,1.0,1,0,...,0.057039,0.230381,0.318473,99.0,0.868916,0.302637,0.138667,0.051933,0.072161,0.000282


Now imagine that we want to match our dual_sqg table with gaia dr3. We can do the same thing, instantiate a new hipscat object with the gaia dr3 table and use the match method to match the sources. 

Luckily some already partitioned catalogs are available at lsdb.io. We can use them to match with our dual catalog.

In [13]:
gaia = lsdb.read_hipscat('https://data.lsdb.io/unstable/gaia_dr3/gaia/')

dual_gaia_sqg = dual_sqg.crossmatch(
    gaia, radius_arcsec=1
)

/Users/gustavoschwarz/miniconda3/lib/python3.12/site-packages/lsdb/dask/crossmatch_catalog_data.py:108: RuntimeWarning: Right catalog does not have a margin cache. Results may be incomplete and/or inaccurate.
  warnings.warn(


Now let's perform the `cone_search` method to get the data we want.

In [15]:
dual_gaia_sqg.cone_search(
    1, #ra
    1, #dec
    1 * 3600 # radius in arcsecs
).compute()

,ID_sqg_sqg_x_dual,RA_sqg_sqg_x_dual,DEC_sqg_sqg_x_dual,CLASS_sqg_sqg_x_dual,model_flag_sqg_sqg_x_dual,PROB_QSO_sqg_sqg_x_dual,PROB_STAR_sqg_sqg_x_dual,PROB_GAL_sqg_sqg_x_dual,Norder_sqg_sqg_x_dual,Dir_sqg_sqg_x_dual,...,ag_gspphot_lower_gaia,ag_gspphot_upper_gaia,ebpminrp_gspphot_gaia,ebpminrp_gspphot_lower_gaia,ebpminrp_gspphot_upper_gaia,libname_gspphot_gaia,Norder_gaia,Dir_gaia,Npix_gaia,_dist_arcsec
_hipscat_index,,,,,,,,,,,,,,,,,,,,,
5091982615334879232,DR4_3_STRIPE82-0003_0027945,1.150865,0.028838,2.0,0,0.0,0.0,1.0,5,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2,0,70,0.228274
5091982615334879233,DR4_3_STRIPE82-0004_0000309,1.150864,0.028841,2.0,0,0.0,0.01,0.99,5,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2,0,70,0.237595
5091984044355223552,DR4_3_STRIPE82-0003_0032189,1.204843,0.022827,2.0,0,0.0,0.0,1.0,5,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2,0,70,0.142705
5091985502148493312,DR4_3_STRIPE82-0004_0002515,1.293757,0.09462,2.0,0,0.0,0.0,1.0,5,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2,0,70,0.242721
5092003219446431744,DR4_3_STRIPE82-0004_0002808,0.923451,0.098841,2.0,1,0.0,0.16,0.84,5,0,...,0.0012,0.0122,0.0028,0.0006,0.0066,MARCS,2,0,70,0.23719
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5477510575233695744,DR4_3_STRIPE82-0004_0032107,1.455972,1.360518,2.0,0,0.0,0.0,1.0,1,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2,0,76,0.17183
5477512731277918208,DR4_3_STRIPE82-0004_0033585,1.315843,1.320673,2.0,0,0.0,0.0,1.0,1,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2,0,76,0.327332
5477522346325573632,DR4_3_STRIPE82-0004_0030245,1.605574,1.412259,2.0,0,0.0,0.0,1.0,1,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2,0,76,0.357235


### Matching an local table with hipscat

Of course you may also load your own table and match with some S-PLUS table.

We may open our table with pandas and then load it into a hipscat object. Then we can use the match method to match the sources.

In [16]:
import pandas as pd

df = pd.read_csv("tables/some_galaxies.csv")
df

,FIELD,ID,RA,DEC,r_petro,zb,zb_Min,zb_Max,PROB_GAL_new,SPIRAL,ELLIPTICAL,UNCERTAIN,simple_class
0,STRIPE82-0001,SPLUS.STRIPE82-0001.03217.griz,359.9707,-1.2178,16.50,0.1118,0.083,0.139,1.0,1,0,0,Ec
1,STRIPE82-0001,SPLUS.STRIPE82-0001.03462.griz,0.1286,-1.2130,15.80,0.0670,0.061,0.072,1.0,0,0,1,SB
2,STRIPE82-0001,SPLUS.STRIPE82-0001.03521.griz,0.0559,-1.2136,16.34,0.0800,0.074,0.079,1.0,1,0,0,SB
3,STRIPE82-0001,SPLUS.STRIPE82-0001.03897.griz,0.2156,-1.1983,16.77,0.2070,0.194,0.221,1.0,0,1,0,Er
4,STRIPE82-0001,SPLUS.STRIPE82-0001.04092.griz,0.1664,-1.1912,16.88,0.1550,0.081,0.158,1.0,0,0,1,Ei
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8913,STRIPE82-0170,SPLUS.STRIPE82-0170.28231.griz,357.9039,0.8854,15.10,0.0740,0.069,0.076,1.0,0,1,0,Er
8914,STRIPE82-0170,SPLUS.STRIPE82-0170.28516.griz,358.1675,0.8765,16.80,0.1230,0.109,0.123,1.0,0,0,1,Er
8915,STRIPE82-0170,SPLUS.STRIPE82-0170.28780.griz,358.2519,0.8705,16.85,0.1350,0.120,0.140,1.0,0,0,1,Er
8916,STRIPE82-0170,SPLUS.STRIPE82-0170.34215.griz,358.3806,1.2268,15.68,0.0710,0.064,0.083,1.0,1,0,0,S


In [17]:
# load it into lsdb
df_hips = lsdb.from_dataframe(df, ra_column="RA", dec_column="DEC", margin_threshold=3600)

Then we can perform the crossmatch with the dual catalog for example.

In [18]:
matched_table = df_hips.crossmatch(dual, radius_arcsec = 1).compute()
matched_table

In [19]:
matched_table.to_csv("tables/some_galaxies_match_dr4dual.csv")